In [21]:
import pandas as pd
import numpy as np 
import os
import csv
import statsmodels.formula.api as smf
import statsmodels.api as sm

anoi = 2022 # especificar o ano inicial a ser estudado
anof = 2024 # especificar o ano final a ser estudado
anos = []
for i in range(anof-anoi+1): # cria uma lista dos anos a serem estudados 
    anos.append(anof-(i))


df = pd.DataFrame()

In [22]:
#Abre todos os arquivos e monta o data frame 

for ano in anos:

    pastadados = f'anac_{ano}'
    nome_arquivos = os.listdir(pastadados)

    Primeiro_Lido = False
    for i in nome_arquivos: 

        with open(pastadados+'/'+str(i), mode = 'r', encoding = 'latin-1') as arq: #descobre o separador do arquivo
            sniff = (csv.Sniffer().sniff(arq.readline())).delimiter

        if not Primeiro_Lido:
            arquivo = pd.read_csv(pastadados+'/'+str(i),sep = sniff, encoding='latin-1', decimal=',', low_memory=False) # Lê o primeiro
            arquivo.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'] #padroniza os arquivos para o mesmo cabeçalho
            Primeiro_Lido = True

        else:
            arquivo_corrido = pd.read_csv(pastadados+'/'+str(i),sep = sniff, encoding='latin-1', decimal=',',low_memory=False) 
        

            while len(arquivo_corrido.columns)>7:
                del arquivo_corrido[f'{arquivo_corrido.columns[0]}'] #retira colunas vazias ou de contagem de linhas

        
                

            arquivo_corrido.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'] #padroniza os arquivos para o mesmo cabeçalho
            arquivo = pd.concat([arquivo, arquivo_corrido]) 

    df = pd.concat([df, arquivo]) 
    

df['TARIFA'] = pd.to_numeric(df['TARIFA'], errors='coerce') # define a coluna tarifa como número e não objeto


df['RECEITA'] = df['TARIFA']*df['ASSENTOS'] #cria coluna de receita


aeroportos_sp_icao = ['SBSP', 'SBGR']
aeroportos_rj_icao = ['SBRJ', 'SBGL']

#filtra a rota RJ-SP, principal rota da aviação comercial (rota trabalhada no projeto)

tabela_filtrada = df[df['ORIGEM'].isin(aeroportos_rj_icao) & df['DESTINO'].isin(aeroportos_sp_icao)]
tabela_filtrado2= df[df['ORIGEM'].isin(aeroportos_sp_icao) & df['DESTINO'].isin(aeroportos_rj_icao)]

dfrota = pd.concat([tabela_filtrada, tabela_filtrado2]) 

#Data Frame principal a ser utilizado
dfrota

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,RECEITA
262299,2024,8,AZU,SBRJ,SBGR,114.64,2,229.28
262300,2024,8,AZU,SBRJ,SBGR,114.65,4,458.60
262301,2024,8,AZU,SBRJ,SBGR,114.66,2,229.32
262302,2024,8,AZU,SBRJ,SBGR,114.67,3,344.01
262303,2024,8,AZU,SBRJ,SBGR,114.70,9,1032.30
...,...,...,...,...,...,...,...,...
407080,2022,3,TAM,SBSP,SBRJ,988.90,2,1977.80
407081,2022,3,TAM,SBSP,SBRJ,995.90,14,13942.60
407082,2022,3,TAM,SBSP,SBRJ,997.90,2,1995.80
407083,2022,3,TAM,SBSP,SBRJ,998.90,7,6992.30


In [27]:
#----------------------------Gráfico 1: Tarifa Média Anual ----------------------------------------------

# monta o df_1 - (Tarifa Media Mensal Por Ano)

df_1 = dfrota.groupby(['ANO','ORIGEM', 'DESTINO', 'EMPRESA', 'MES'])[['RECEITA', 'ASSENTOS']].sum()
df_1['TARIFA_MEDIA_MENSAL'] = df_1['RECEITA'] / df_1['ASSENTOS']


df_1 = df_1.reset_index()[['ANO', 'MES','EMPRESA', 'ORIGEM', 'DESTINO' ,'TARIFA_MEDIA_MENSAL' ]]

df_1['ANO'] = df_1['ANO'].astype(int)
df_1['MES'] = df_1['MES'].astype(int)   

df_1

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA_MEDIA_MENSAL
0,2022,5,AZU,SBGL,SBGR,953.328571
1,2022,9,AZU,SBGL,SBGR,2001.600000
2,2022,11,AZU,SBGL,SBGR,343.800000
3,2022,12,AZU,SBGL,SBGR,2158.150000
4,2022,1,GLO,SBGL,SBGR,505.261414
...,...,...,...,...,...,...
788,2024,8,TAM,SBSP,SBRJ,439.809928
789,2024,9,TAM,SBSP,SBRJ,429.933877
790,2024,10,TAM,SBSP,SBRJ,530.048853
791,2024,11,TAM,SBSP,SBRJ,459.576693


In [24]:
#Arrumando o dataset
combu = pd.read_excel('combustiveis-aviacao-regioes.xlsx')

combustiveis = combu[combu['REGIÃO'] == 'SUDESTE']

querosene = combustiveis[combustiveis['PRODUTO'] == 'QUEROSENE DE AVIAÇÃO']

querosene['MÊS'] = pd.to_datetime(querosene['MÊS']) #Formata a coluna da data, seperando em ano e mes 
querosene['ANO'] = querosene['MÊS'].dt.year
querosene['MES'] = querosene['MÊS'].dt.month

qav = querosene[['ANO', 'MES', 'UNIDADE DE MEDIDA', 'PREÇO MÉDIO DE DISTRIBUIÇÃO']]

qav

/var/folders/9k/3vndh6qd4jv3p0mmd8shw2vr0000gn/T/ipykernel_15203/1389388272.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  querosene['MÊS'] = pd.to_datetime(querosene['MÊS']) #Formata a coluna da data, seperando em ano e mes
/var/folders/9k/3vndh6qd4jv3p0mmd8shw2vr0000gn/T/ipykernel_15203/1389388272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  querosene['ANO'] = querosene['MÊS'].dt.year
/var/folders/9k/3vndh6qd4jv3p0mmd8shw2vr0000gn/T/ipykernel_15203/1389388272.py:10: SettingWithCopyWarning: 
A 

,ANO,MES,UNIDADE DE MEDIDA,PREÇO MÉDIO DE DISTRIBUIÇÃO
581,2018,11,R$/l,3.492233
582,2018,12,R$/l,3.220913
583,2019,1,R$/l,2.903441
584,2019,2,R$/l,2.841723
585,2019,3,R$/l,2.942767
...,...,...,...,...
659,2025,5,R$/l,4.471988
660,2025,6,R$/l,4.147937
661,2025,7,R$/l,4.281472
662,2025,8,R$/l,4.461266


In [25]:
# Supondo que o segundo dataframe se chame df_combustivel
# e tenha colunas: ['ANO', 'MES', 'PREÇO MÉDIO DE DISTRIBUIÇÃO']

df_f = df_1.merge(
    qav[['ANO', 'MES', 'PREÇO MÉDIO DE DISTRIBUIÇÃO']],
    on=['ANO', 'MES'],
    how='left'
)

df_f.rename(columns={'PREÇO MÉDIO DE DISTRIBUIÇÃO': 'PRECO_MEDIO_DISTRIBUICAO'}, inplace=True)

df_f

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA_MEDIA_MENSAL,PRECO_MEDIO_DISTRIBUICAO
0,2022,5,AZU,SBGL,SBGR,953.328571,6.123784
1,2022,9,AZU,SBGL,SBGR,2001.600000,6.198058
2,2022,11,AZU,SBGL,SBGR,343.800000,6.506513
3,2022,12,AZU,SBGL,SBGR,2158.150000,6.118435
4,2022,1,GLO,SBGL,SBGR,505.261414,4.434005
...,...,...,...,...,...,...,...
788,2024,8,TAM,SBSP,SBRJ,439.809928,4.948092
789,2024,9,TAM,SBSP,SBRJ,429.933877,4.716770
790,2024,10,TAM,SBSP,SBRJ,530.048853,4.316450
791,2024,11,TAM,SBSP,SBRJ,459.576693,4.386880
